In this notebook we use GPU resources to train a convolutional neural network on satellite images of the amazon rainforest. This is a multiclass problem where one image can be tagged with several classes.

This notebook is run in the Google Colab environment due to the large size of the dataset. For this particular (initial) iteration of the task I significantly reduced the size of the dataset (only using 10% of the training data folder and then split it further into a train and test set).

The CNN consists of different convolutional/pooling layers without much research (for now) behind why it looks like it does. The model´s training and validation accuracy could be a lot better so we will try to work on that in the next iteration of this research project.

In [1]:
from tensorflow.keras import layers, models
import tensorflow as tf
import os
from google.colab import drive
import matplotlib.pyplot as plt
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pandas as pd


In [30]:
device = 'cpu'
if tf.test.is_gpu_available(cuda_only=True):
    device = 'cuda'
print("Using device:", device)

Using device: cuda


In [5]:
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
# Define the path to the image folder
train_folder = '/content/drive/MyDrive/2024/data/amazon_data/train-jpg'
test_folder = '/content/drive/MyDrive/2024/data/amazon_data/test-jpg'

In [8]:
# Set up the data generator for your dataset directory
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Scale pixel values, and split for validation

In [9]:
# Create a directory for Kaggle API
os.makedirs('/root/.kaggle', exist_ok=True)

# Copy the kaggle.json file to the appropriate directory
!cp /content/drive/MyDrive/2024/data/kaggle.json /root/.kaggle/

# Set permissions
!chmod 600 /root/.kaggle/kaggle.json

# Download a Kaggle competition dataset
!kaggle competitions download -c planet-understanding-the-amazon-from-space

# Unzip the Kaggle dataset
!unzip -q planet-understanding-the-amazon-from-space.zip -d /content/planet_dataset/

  0% 0.00/2.94M [00:00<?, ?B/s]
100% 2.94M/2.94M [00:00<00:00, 82.0MB/s]


In [10]:
#the train folder contains 40k images, I think that is enough for training and testing
#I also did not find a csv file for the test set such as the train_v2.csv that is available for the train set

df = pd.read_csv('/content/planet_dataset/train_v2.csv/train_v2.csv')

df['full_image_path'] = train_folder + '/' + df['image_name'] + '.jpg'

print(df)

        image_name                                           tags  \
0          train_0                                   haze primary   
1          train_1                agriculture clear primary water   
2          train_2                                  clear primary   
3          train_3                                  clear primary   
4          train_4      agriculture clear habitation primary road   
...            ...                                            ...   
40474  train_40474                                  clear primary   
40475  train_40475                                         cloudy   
40476  train_40476                      agriculture clear primary   
40477  train_40477                 agriculture clear primary road   
40478  train_40478  agriculture cultivation partly_cloudy primary   

                                         full_image_path  
0      /content/drive/MyDrive/2024/data/amazon_data/t...  
1      /content/drive/MyDrive/2024/data/amazon_data/t

In [12]:
a = df['tags'].unique()

unique_tags_combo  = ' '.join(a)

unique_tags = set(unique_tags_combo.split(' '))

print(unique_tags)

{'blooming', 'road', 'cultivation', 'bare_ground', 'conventional_mine', 'selective_logging', 'slash_burn', 'primary', 'water', 'cloudy', 'partly_cloudy', 'habitation', 'artisinal_mine', 'blow_down', 'clear', 'haze', 'agriculture'}


In [13]:
# Split the labels into lists
df['tags'] = df['tags'].str.split()

# Get unique classes
# Flatten the list of labels
unique_classes = set(class_ for sublist in df['tags'] for class_ in sublist)

# Create binary columns for each unique class
for cls in unique_classes:
    df[cls] = df['tags'].apply(lambda x: 1 if cls in x else 0)

# Drop the original labels column if you no longer need it
df = df.drop(columns=['tags'])

In [14]:
#downsampled the dataset for performance purposes
df_downsampled = df.sample(frac=0.1, random_state=42)

In [15]:
df_downsampled

,image_name,full_image_path,blooming,road,cultivation,bare_ground,conventional_mine,selective_logging,slash_burn,primary,water,cloudy,partly_cloudy,habitation,artisinal_mine,blow_down,clear,haze,agriculture
34602,train_34602,/content/drive/MyDrive/2024/data/amazon_data/t...,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0
11243,train_11243,/content/drive/MyDrive/2024/data/amazon_data/t...,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0
14499,train_14499,/content/drive/MyDrive/2024/data/amazon_data/t...,0,1,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0
18918,train_18918,/content/drive/MyDrive/2024/data/amazon_data/t...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
30631,train_30631,/content/drive/MyDrive/2024/data/amazon_data/t...,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1748,train_1748,/content/drive/MyDrive/2024/data/amazon_data/t...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0
14812,train_14812,/content/drive/MyDrive/2024/data/amazon_data/t...,0,1,0,0,0,0,0,1,1,0,0,1,0,0,1,0,0
21912,train_21912,/content/drive/MyDrive/2024/data/amazon_data/t...,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0
16200,train_16200,/content/drive/MyDrive/2024/data/amazon_data/t...,0,0,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0


In [16]:
# Split train and validation
df_train = df_downsampled.sample(frac=0.7, random_state=42)
df_test = df_downsampled.drop(df_train.index)

In [17]:
#preprocess images to be of a certain size (128, 128) and normalize them to be in the range [0,1]
def process_image(file_path, label):
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=3)  # Adjust based on image format
    img = tf.image.resize(img, [128, 128])  # Resize to match your input size
    img /= 255.0  # Normalize to [0,1] range
    return img, label

In [21]:
labels_train = df_train[df_train.columns.drop(['image_name', 'full_image_path'])].values
labels_test = df_test[df_test.columns.drop(['image_name', 'full_image_path'])].values

In [22]:
# Create TensorFlow Dataset

#create a (image path, label) tuple
train_ds = tf.data.Dataset.from_tensor_slices((df_train['full_image_path'].values, labels_train))
#call the process_image fn to load the image from its path, resize it and normalise the pixel values to return a (processed image, label pair)
#we apply parallelism and choose AUTOTUNE so that tensorflow decides on the optimal level of parallelism to speed up data loading
train_ds = train_ds.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
#One batch or iteration is set to 32 images (meaning that the model parameters are updated after 32 images)
#We pull 1000 pictures into a buffer memory which is shuffled before training and then batches of 32 are pulled from this buffer
#prefetch is when we load the next batch while training the current one to reduce idle time
train_ds = train_ds.batch(32).shuffle(buffer_size=1000).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((df_test['full_image_path'].values, labels_test))
val_ds = val_ds.map(process_image, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(32).prefetch(tf.data.AUTOTUNE)


In [26]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# Add additional layers as needed
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))  # Optional hidden layer
model.add(layers.Dense(17, activation='sigmoid'))  # 17 output units for 17 classes

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [27]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 28, 28, 64)          │          36,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 50176)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │       3,211,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 17)                  │           1,105 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 3,268,753 (12.47 MB)

 Trainable params: 3,268,753 (12.47 MB)

 Non-trainable params: 0 (0.00 B)

In [28]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [29]:
model.fit(train_ds, epochs=15, validation_data=val_ds,verbose=2)

Epoch 1/15


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:681: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


89/89 - 554s - 6s/step - accuracy: 0.6507 - loss: 0.6758 - val_accuracy: 0.6334 - val_loss: 0.6588
Epoch 2/15
89/89 - 15s - 166ms/step - accuracy: 0.6581 - loss: 0.6423 - val_accuracy: 0.6334 - val_loss: 0.6272
Epoch 3/15
89/89 - 15s - 165ms/step - accuracy: 0.6581 - loss: 0.6115 - val_accuracy: 0.6334 - val_loss: 0.5981
Epoch 4/15
89/89 - 20s - 225ms/step - accuracy: 0.6581 - loss: 0.5833 - val_accuracy: 0.6334 - val_loss: 0.5714
Epoch 5/15
89/89 - 18s - 202ms/step - accuracy: 0.6581 - loss: 0.5573 - val_accuracy: 0.6334 - val_loss: 0.5470
Epoch 6/15
89/89 - 13s - 144ms/step - accuracy: 0.6581 - loss: 0.5335 - val_accuracy: 0.6334 - val_loss: 0.5245
Epoch 7/15
89/89 - 14s - 152ms/step - accuracy: 0.6581 - loss: 0.5116 - val_accuracy: 0.6334 - val_loss: 0.5040
Epoch 8/15
89/89 - 15s - 169ms/step - accuracy: 0.6581 - loss: 0.4916 - val_accuracy: 0.6334 - val_loss: 0.4852
Epoch 9/15
89/89 - 13s - 143ms/step - accuracy: 0.6581 - loss: 0.4732 - val_accuracy: 0.6334 - val_loss: 0.4679
Epoch